# Instrument Classification

## Downloading the python packages

In [4]:
# Download python packages required using pip
!python -m pip install --upgrade pip
!pip3 install torch torchaudio
!pip install torch-summary
!pip install pandas pyarrow
!pip install librosa
!pip install tqdm

In [5]:
# Import python packages required
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchaudio
from torchsummary import summary
import pandas as pd
import os
from tqdm import tqdm
import librosa
import numpy as np
from IPython.display import display

## Downloading the dataset

The IRMAS dataset is downloaded to Google Drive and then unzipped into the local colab session.

In [20]:
# Defining the path constants used
AUDIO_FILES = 'wavfiles'

## Annotation File

The code below creates an annotations file which provides the information for the dataset including the filenames and class labels.

In [21]:
def create_file_annotations():
  data = []

  # Walking through the directories in the dataset
  for (dirpath, dirnames, filenames) in os.walk(AUDIO_FILES):
    for dir in dirnames:
      for (dirpath, dirnames, filenames) in os.walk(os.path.join(AUDIO_FILES, dir)):
        for filename in filenames:
          data.append([filename, dir])
    break

  # Creating a pandas dataframe to hold this data
  df = pd.DataFrame(data, columns =['Filename', 'ClassLabel'])

  # Create class ID numbers from the class labels
  df['ClassID'] = df.groupby(['ClassLabel']).ngroup()

  return df

annotations = create_file_annotations()

# Display the dataframe
display(annotations)

annotations.set_index('Filename', inplace=True)
for record in annotations.index:
  signal, fs = librosa.load(os.path.join(AUDIO_FILES, record), mono=True, sr=None)
  annotations.at[record, 'Length'] = signal.shape[0]/fs

classes = list(np.unique(annotations.label))
class_dist = annotations.groupby(['ClassLabel'])['Length'].mean()

# Save the dataframe to a CSV file
annotations.to_csv(path_or_buf="annotations.csv", sep=',', encoding='utf-8', index=False)

,Filename,ClassLabel,ClassID
0,0356dec7.wav,Acoustic_guitar,0
1,0eeaebcb.wav,Acoustic_guitar,0
2,10da16fb.wav,Acoustic_guitar,0
3,160c92d7.wav,Acoustic_guitar,0
4,16c9a838.wav,Acoustic_guitar,0
...,...,...,...
295,d6144e01.wav,Violin_or_fiddle,9
296,d84c8c43.wav,Violin_or_fiddle,9
297,d914225a.wav,Violin_or_fiddle,9
298,e3f479f3.wav,Violin_or_fiddle,9


C:\Users\hfret\AppData\Local\Temp\ipykernel_11648\2206804438.py:27: UserWarning: PySoundFile failed. Trying audioread instead.
  signal, fs = librosa.load(os.path.join(AUDIO_FILES, record), mono=True, sr=None)
c:\Users\hfret\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'wavfiles\\0356dec7.wav'

## Custom Dataset

The custom dataset imports the audio dataset's annotation file into python and uses it to load the correct audio files with the corresponding class labels.

In [ ]:
class AudioDataset(Dataset):
  ''' Custom Dataset class for audio instrument classification '''

  def __init__(self, annotation_file, audio_dir, device, transformation, transform_fs, num_samples):
    '''
    Parameters -
        annotation_file: a path to a csv file with the annotations for the dataset
                            these annotations should have the filename at index 0 and class ID at -1
        audio_dir:       the path to the audio dataset directory
        device:          the device in use (cude or cpu)
        transformation:  provides the function for performing preprocessing on the data
        transform_fs:    the samplerate to resample at
        num_samples:     the length n of samples to cut / pad the data to
    '''
    # Read in the annotation file for the dataset
    self.annotations = pd.read_csv(annotation_file)

    # Defining attributes
    self.audio_dir = audio_dir
    self.device = device
    self.transformation = transformation.to(self.device) # putting the data onto a cuda device is available
    self.transform_fs = transform_fs
    self.num_samples = num_samples

    # Creating a unique ID for the classes
    self.annotations.assign(id=self.annotations.groupby(['Instrument']).ngroup())

  def __len__(self):
    ''' Magic method to provide the length of the object '''
    return len(self.annotations)

  def __getitem__(self, index):
    ''' Magic method to provide indexing for the object '''

    # Gets the audio path and labelID for the index
    audio_sample_path, labelID = self._get_audio_sample_path_and_label(index)

    # Loads the audio input to the device
    signal, fs = torchaudio.load(audio_sample_path, normalize=True)
    signal = signal.to(self.device)

    # Resamples and reshapes the audio
    signal = self._resample_audio(signal, fs)
    signal = self._reshape_audio(signal)

    # Performs transformation on the device
    signal = self.transformation(signal)

    return signal, labelID

  def get_class_labels(self):
    ''' Public method to provide a list of the class labels '''
    return self.annotations['Instrument'].unique()

  def _get_audio_sample_path_and_label(self, index):
      ''' Private get method for the sample path location
                        and prediction labelID            '''
      label = self.annotations.iloc[index, 2]
      labelID = self.annotations.iloc[index, 7]
      path = os.path.join(self.audio_dir, label, self.annotations.iloc[index, 0])
      return path, labelID

  def _resample_audio(self, signal, fs):
      # Resample the audio signal if needed
      if fs != self.transform_fs:
          resampler = torchaudio.transforms.Resample(fs, self.transform_fs).to(self.device)
          signal = resampler(signal)
      return signal

  def _reshape_audio(self, signal):
      # Convert the signal to mono if needed
      if signal.shape[0] > 1:
          signal = torch.mean(signal, dim=0, keepdim=True)

      # Cut the signal if needed
      if signal.shape[1] > self.num_samples:
          signal = signal[:, :self.num_samples]

      # Pad the signal if needed
      if signal.shape[1] < self.num_samples:
          signal = torch.nn.functional.pad(signal, (0, self.num_samples - signal.shape[1]))

      return signal

# Defining the dataset and dataloader constants
SAMPLE_RATE = 22050
NUM_SAMPLES = 22050
BATCH_SIZE = 128

# Creating a mel spectogram for the feature transformation
mel_spectogram = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=1024,
    hop_length=512,
    n_mels=64
)

# Defining the device being used
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Device available: {device.upper()}")

# Creating the dataset and dataloader
audio_dataset = AudioDataset(f"{LOCAL_IRMAS}.csv", LOCAL_IRMAS, device, mel_spectogram, SAMPLE_RATE, NUM_SAMPLES)
data_loader = DataLoader(audio_dataset, batch_size=BATCH_SIZE)

# Printing all the class labels
print(audio_dataset.get_class_labels())
# 0 cel
# 1 cla
# 2 flu
# 3 gac
# 4 gel
# 5 org
# 6 pia
# 7 sax
# 8 tru
# 9 vio
# 10 voi

Device available: CUDA
['flu' 'cel' 'org' 'cla' 'pia' 'sax' 'gac' 'gel' 'voi' 'tru' 'vio']


## Convolutional Neural Network

Defining the convolutional neural network architecture with the layers and activation functions.

In [ ]:
class CNNNetwork(nn.Module):
  ''' Custom Convolution Neural Network for audio classification'''

  def __init__(self, num_classes):
    '''
    Parameters -
        num_classes: the number of classes used for classification
    '''
    super().__init__()

    # Define convolutional blocks
    self.conv1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv3 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.conv4 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    # Create a flatten layer to reshape the data into a 1D vector
    self.flatten = nn.Flatten()

    # Create a linear / dense layer to classify the data from the convolutional network
    self.linear = nn.Linear(in_features=128*5*4, out_features=num_classes)

    # Create a softmax layer to provide decimal probabilities to the class predictions
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_data):
    ''' Passing the data through the network '''
    x = self.conv1(input_data)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.conv4(x)
    x = self.flatten(x)
    logits = self.linear(x)
    predictions = self.softmax(logits)
    return predictions

# Creating the neural network
cnn = CNNNetwork(audio_dataset.get_class_labels().size).to(device)

# Printing a summary of the network
summary(cnn.cuda(), (1, 64, 44))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 66, 46]             160
              ReLU-2           [-1, 16, 66, 46]               0
         MaxPool2d-3           [-1, 16, 33, 23]               0
            Conv2d-4           [-1, 32, 35, 25]           4,640
              ReLU-5           [-1, 32, 35, 25]               0
         MaxPool2d-6           [-1, 32, 17, 12]               0
            Conv2d-7           [-1, 64, 19, 14]          18,496
              ReLU-8           [-1, 64, 19, 14]               0
         MaxPool2d-9             [-1, 64, 9, 7]               0
           Conv2d-10           [-1, 128, 11, 9]          73,856
             ReLU-11           [-1, 128, 11, 9]               0
        MaxPool2d-12            [-1, 128, 5, 4]               0
          Flatten-13                 [-1, 2560]               0
           Linear-14                   

## Training

Below demonstrates the function to train the neural network. This then saves the model state to google drive.

In [ ]:
def train(model, data_loader, loss_func, optimiser, device, epochs):
  '''
  Parameters -
    model: the model to train
    data_loader: the dataloader to optimise memory storage with batch downloads
    loss_func: the loss function to use to update the bias weightings
    optimiser: the optimiser to use for backwards propogation
    device: the device in use (cuda or cpu)
    epochs: the number of epochs to run
  '''
  model.train()

  # Using a loading bar to show progress
  with tqdm(data_loader, unit="batch", total=len(data_loader)) as tepoch:

    # Increament through each epoch
    for epoch in range(epochs):
      for input, target in data_loader:
        tepoch.set_description(f"Epoch {epoch}")

        # Load the batch input into the device memory
        input, target = input.to(device), target.to(device)

        # Calculate the loss
        output = model(input) # pass the inputs into the model
        loss = loss_func(output, target) # compare the predictions to the actual data

        # Calculate the accuracy of the model to display
        predictions = output.argmax(dim=1, keepdim=True).squeeze()
        correct = (predictions == target).sum().item()
        accuracy = correct / BATCH_SIZE

        # Backpropogate the loss and update the NN weights
        optimiser.zero_grad() # resets the gradients
        loss.backward() # performs back propogation
        optimiser.step() # updates the weights

        tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)

  print("\nTraining Completed")

# Defining network constants
EPOCHS = 40 # [estimated 1.5 mins / per epoch]
LEARNING_RATE = .001

# Defining the loss function
loss_func = nn.CrossEntropyLoss()

# Defining the optimiser
optimiser = torch.optim.Adam(cnn.parameters(), lr=LEARNING_RATE)

# Training the model
train(cnn, data_loader, loss_func, optimiser, device, EPOCHS)

# Saving the model state
model_state_path = os.path.join(GDRIVE_AUDIO, "cnn.pth")
torch.save(cnn.state_dict(), model_state_path)
print(f"Model trained and stored  at '{model_state_path}'")

Epoch 39:   0%|          | 0/53 [1:03:31<?, ?batch/s, accuracy=38.3, loss=1.54]


Training Completed
Model trained and stored  at '/content/drive/MyDrive/Audio/cnn.pth'


## References

<Bosch2012>Bosch, J. J., Janer, J., Fuhrmann, F., & Herrera, P. (2012). "A Comparison of Sound Segregation Techniques for Predominant Instrument Recognition in Musical Audio Signals." In Proceedings of the International Society for Music Information Retrieval Conference (ISMIR) (pp. 559-564).